<a href="https://colab.research.google.com/github/myeze/WhisperBuildTrimmer/blob/main/PyannoteWhisperLocal%26Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/ArthurFDLR/whisper-youtube/blob/main/whisper_youtube.ipynb)](https://colab.research.google.com/github/Majdoddin/nlp/blob/main/Pyannote_plays_and_Whisper_rhymes_v_2_0.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/openai/whisper)](https://github.com/majdoddin/nlp)

# Whisper's transcription plus Pyannote's Diarization

**Update** - [@johnwyles](https://github.com/johnwyles) added HTML output for audio/video files from Google Drive, along with some fixes.

Using the new word-level timestamping of Whisper, the transcription words are highlighted as the video plays, with optional autoscroll. And the display on small displays is improved.

Moreover, the model is loaded just once, thus the whole thing runs much faster now. You can also hardcode your Huggingface token.

---
Andrej Karpathy [suggested](https://twitter.com/karpathy/status/1574476200801538048?s=20&t=s5IMMXOYjBI6-91dib6w8g) training a classifier on top of  OpenAI [Whisper](https://openai.com/blog/whisper/) model features to identify the speaker, so we can visualize the speaker in the transcript. But, as [pointed out](https://twitter.com/tarantulae/status/1574493613362388992?s=20&t=s5IMMXOYjBI6-91dib6w8g) by Christian Perone, it seems that features from whisper wouldn't be that great for speaker recognition as its main objective is basically to ignore speaker differences.

In the following, I use [**`pyannote-audio`**](https://github.com/pyannote/pyannote-audio), a speaker diarization toolkit by Hervé Bredin, to identify the speakers, and then match it with the transcriptions of Whispr, linked to the video. The input can be YouTube or an video/audio file (also on Google Drive). I try it on a [Customer Support Call](https://youtu.be/hpZFJctBUHQ). Check the result [**here**](https://majdoddin.github.io/dyson.html).

To make it easier to match the transcriptions to diarizations by speaker change, Sarah Kaiser [suggested](https://github.com/openai/whisper/discussions/264#discussioncomment-3825375) runnnig the pyannote.audio first and  then just running whisper on the split-by-speaker chunks.
For sake of performance (and transcription quality?), we attach the audio segments into a single audio file with a silent spacer as a separator, and run whisper on it. Enjoy it!

(For sake of performance , I also tried attaching the audio segments into a single audio file with a silent -or beep- spacer as a separator, and run whisper on it see it on [colab](https://colab.research.google.com/drive/1HuvcY4tkTHPDzcwyVH77LCh_m8tP-Qet?usp=sharing). It [works](https://majdoddin.github.io/lexicap.html) on some audio, and fails on some (Dyson's Interview). The problem is, whisper does not reliably make a timestap on a spacer. See the discussions [#139](https://github.com/openai/whisper/discussions/139) and [#29](https://github.com/openai/whisper/discussions/29))

The Markdown form used below is from [@ArthurFDLR](https://github.com/ArthurFDLR/whisper-youtube/).   

# Preparing the audio file

**Optional:** Mount Google Drive



In [ ]:
from google.colab import drive
from pathlib import Path

drive_mount_path = Path("/content/drive")
drive.mount(str(drive_mount_path))
drive_mount_path /= "MyDrive"

In [ ]:
from pathlib import Path
#@markdown Enter the URL of the YouTube video, or the path to the video/audio file you want to transcribe, give the output path, etc. and run the cell. HTML file embeds the video for YouTube, and audio for media files.

Source = 'File (Google Drive)' #@param ['Youtube', 'File (Google Drive)']
#@markdown ---
#@markdown #### **Youtube video**
video_url = "https://youtu.be/hpZFJctBUHQ" #@param {type:"string"}
#store_audio = True #@param {type:"boolean"}
#@markdown ---
#@markdown #### **Google Drive video or audio path (mp4, wav, mp3)**
video_path = "/content/drive/MyDrive/Audio/Interview1.mp3" #@param {type:"string"}
#@markdown ---
output_path = "/content/transcript/" #@param {type:"string"}
output_path = str(Path(output_path))
#@markdown ---
#@markdown #### **Title for transcription of media file**
audio_title = "Sample Order Taking" #@param {type:"string"}
#@markdown ---
#@markdown #### Copy a token from your [Hugging Face tokens page](https://huggingface.co/settings/tokens) and paste it below.
access_token = "hf_" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change the video.**

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
Path(output_path).mkdir(parents=True, exist_ok=True)
%cd {output_path}
video_title = ""
video_id = ""

/content/transcript


## From YouTube

 Installing [`yt-dlp`](https://github.com/yt-dlp/yt-dlp) and downloading the [video](https://youtu.be/NSp2fEQ6wyA) from youtube.

In [ ]:
if Source == "Youtube":
  !pip install -U yt-dlp

Custom build of `ffmpeg` as [recommended](https://github.com/yt-dlp/yt-dlp#strongly-recommended) by `yt-dlp`.

In [ ]:
if Source == "Youtube":
  !wget -O - -q  https://github.com/yt-dlp/FFmpeg-Builds/releases/download/latest/ffmpeg-master-latest-linux64-gpl.tar.xz | xz -qdc| tar -x

In [ ]:
#Getting video info
if Source == "Youtube":
  from yt_dlp import YoutubeDL
  with YoutubeDL() as ydl:
    info_dict = ydl.extract_info(video_url, download=False)
    video_title = info_dict.get('title', None)
    video_id = info_dict.get('id', None)
    print("Title: " + video_title) # <= Here, you got the video title


Downloading the audio from YouTube.

In [ ]:
if Source == "Youtube":
  !yt-dlp -xv --ffmpeg-location ffmpeg-master-latest-linux64-gpl/bin --audio-format wav  -o "{str(output_path) + '/'}input.wav" -- {video_url}

## or from File (Google Drive)

In [ ]:
if Source == 'File (Google Drive)':
    !ffmpeg -i {repr(video_path)} -vn -acodec pcm_s16le -ar 16000 -ac 1 -y input.wav
print(len(video_path))

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

## Prepending a spacer

`pyannote.audio` seems to miss the first 0.5 seconds of the audio, and, therefore, we prepend a spcacer.

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment

spacermilli = 2000
spacer = AudioSegment.silent(duration=spacermilli)


audio = AudioSegment.from_wav("input.wav")

audio = spacer.append(audio, crossfade=0)

audio.export('input_prep.wav', format='wav')

<_io.BufferedRandom name='input_prep.wav'>

# Pyannote's Diarization

[`pyannote.audio`](https://github.com/pyannote/pyannote-audio) is an open-source toolkit written in Python for **speaker diarization**.

Based on [`PyTorch`](https://pytorch.org) machine learning framework, it provides a set of trainable end-to-end neural building blocks that can be combined and jointly optimized to build speaker diarization pipelines.

`pyannote.audio` also comes with pretrained [models](https://huggingface.co/models?other=pyannote-audio-model) and [pipelines](https://huggingface.co/models?other=pyannote-audio-pipeline) covering a wide range of domains for voice activity detection, speaker segmentation, overlapped speech detection, speaker embedding reaching state-of-the-art performance for most of them.

Installing `pyannote.audio`.

In [ ]:
!pip install light-the-torch

In [ ]:
!ltt install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1

  Using cached https://download.pytorch.org/whl/cu117/torch-1.13.1%2Bcu117-cp310-cp310-linux_x86_64.whl (1801.8 MB)
  Using cached https://download.pytorch.org/whl/cu117/torchaudio-0.13.1%2Bcu117-cp310-cp310-linux_x86_64.whl (4.2 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1
    Uninstalling torch-2.3.1:
      Successfully uninstalled torch-2.3.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.0
    Uninstalling torchaudio-2.4.0:
      Successfully uninstalled torchaudio-2.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lightning 2.4.0 requires torch<4.0,>=2.1.0, but you have torch 1.13.1+cu117 which is incompatible.
openai-whisper 20231117 requires triton<3,>=2.0.0; platform_machine == "x86_64" and sys_platform == "linux" or sys_platform == "linux2", but you have triton 3.0.0 which is inco

In [ ]:
!pip install  git+https://github.com/hmmlearn/hmmlearn.git
!pip install  git+https://github.com/pyannote/pyannote-audio.git@develop

  Cloning https://github.com/hmmlearn/hmmlearn.git to /tmp/pip-req-build-6ks7nydm
  Running command git clone --filter=blob:none --quiet https://github.com/hmmlearn/hmmlearn.git /tmp/pip-req-build-6ks7nydm
  Resolved https://github.com/hmmlearn/hmmlearn.git to commit 33b1a916a11efb5c569abfc9705b7ae030e81c31
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/pyannote/pyannote-audio.git (to revision develop) to /tmp/pip-req-build-v1kadimi
  Running command git clone --filter=blob:none --quiet https://github.com/pyannote/pyannote-audio.git /tmp/pip-req-build-v1kadimi
  Resolved https://github.com/pyannote/pyannote-audio.git to commit 286ea1a4e34e2dd7d7926f590e402dac1e17494b
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for torch>=2.0.0 from https://files.pythonhosted.org/packages/9a/bd/4161ae28fb1c388a8ee30ca3aa72cf11ac3016ce62bc9e82c71ce193c410/torch-2.4.0-cp310-cp310-manylinux1_

**Important:** To load the pyannote speaker diarization pipeline,

* accept the user conditions on both [hf.co/pyannote/speaker-diarization](https://hf.co/pyannote/speaker-diarization) and [hf.co/pyannote/segmentation](https://huggingface.co/pyannote/segmentation).
* paste your access_token or login using `notebook_login` below

In [ ]:
if not(access_token):
    from huggingface_hub import notebook_login
    notebook_login()

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization', use_auth_token= (access_token) or True )

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline.to(device)

Running pyannote.audio to generate the diarizations.

In [ ]:
DEMO_FILE = {'uri': 'blabla', 'audio': 'input_prep.wav'}
dz = pipeline(DEMO_FILE)

with open("diarization.txt", "w") as text_file:
    text_file.write(str(dz))

In [ ]:
print(*list(dz.itertracks(yield_label = True))[:10], sep="\n")

(<Segment(3.79409, 5.07659)>, 'A', 'SPEAKER_04')
(<Segment(5.07659, 5.12722)>, 'B', 'SPEAKER_05')
(<Segment(6.52784, 30.7435)>, 'C', 'SPEAKER_05')
(<Segment(14.6953, 14.9822)>, 'D', 'SPEAKER_04')
(<Segment(26.5247, 26.5922)>, 'E', 'SPEAKER_07')
(<Segment(26.5922, 26.626)>, 'F', 'SPEAKER_06')
(<Segment(31.351, 38.506)>, 'G', 'SPEAKER_05')
(<Segment(39.2653, 40.666)>, 'H', 'SPEAKER_05')
(<Segment(41.2566, 74.3316)>, 'I', 'SPEAKER_05')
(<Segment(56.3428, 56.6128)>, 'J', 'SPEAKER_07')


# Preparing audio files according to the diarization

In [ ]:
def millisec(timeStr):
  spl = timeStr.split(":")
  s = (int)((int(spl[0]) * 60 * 60 + int(spl[1]) * 60 + float(spl[2]) )* 1000)
  return s

Grouping the diarization segments according to the speaker.

In [ ]:
import re
dzs = open('diarization.txt').read().splitlines()

groups = []
g = []
lastend = 0

for d in dzs:
  if g and (g[0].split()[-1] != d.split()[-1]):      #same speaker
    groups.append(g)
    g = []

  g.append(d)

  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=d)[1]
  end = millisec(end)
  if (lastend > end):       #segment engulfed by a previous segment
    groups.append(g)
    g = []
  else:
    lastend = end
if g:
  groups.append(g)
print(*groups, sep='\n')

['[ 00:00:03.794 -->  00:00:05.076] A SPEAKER_04']
['[ 00:00:05.076 -->  00:00:05.127] B SPEAKER_05', '[ 00:00:06.527 -->  00:00:30.743] C SPEAKER_05']
['[ 00:00:14.695 -->  00:00:14.982] D SPEAKER_04']
['[ 00:00:26.524 -->  00:00:26.592] E SPEAKER_07']
['[ 00:00:26.592 -->  00:00:26.625] F SPEAKER_06']
['[ 00:00:31.350 -->  00:00:38.505] G SPEAKER_05', '[ 00:00:39.265 -->  00:00:40.665] H SPEAKER_05', '[ 00:00:41.256 -->  00:01:14.331] I SPEAKER_05']
['[ 00:00:56.342 -->  00:00:56.612] J SPEAKER_07']
['[ 00:00:57.034 -->  00:00:57.979] K SPEAKER_06']
['[ 00:01:15.158 -->  00:01:18.465] L SPEAKER_01']
['[ 00:01:17.436 -->  00:01:18.735] M SPEAKER_05']
['[ 00:01:21.267 -->  00:01:22.988] N SPEAKER_01', '[ 00:01:23.595 -->  00:01:33.754] O SPEAKER_01']
['[ 00:01:29.029 -->  00:01:29.215] P SPEAKER_00']
['[ 00:01:30.244 -->  00:01:30.531] Q SPEAKER_00']
['[ 00:01:30.531 -->  00:01:31.746] R SPEAKER_07']
['[ 00:01:34.514 -->  00:01:37.467] S SPEAKER_01', '[ 00:01:38.817 -->  00:01:42.917] 

Save the audio part corresponding to each diarization group.

In [ ]:
audio = AudioSegment.from_wav("input_prep.wav")
gidx = -1
for g in groups:
  start = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  end = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[-1])[1]
  start = millisec(start) #- spacermilli
  end = millisec(end)  #- spacermilli
  gidx += 1
  audio[start:end].export(str(gidx) + '.wav', format='wav')
  print(f"group {gidx}: {start}--{end}")

group 0: 3794--5076
group 1: 5076--30743
group 2: 14695--14982
group 3: 26524--26592
group 4: 26592--26625
group 5: 31350--74331
group 6: 56342--56612
group 7: 57034--57979
group 8: 75158--78465
group 9: 77436--78735
group 10: 81267--93753
group 11: 89029--89215
group 12: 90244--90531
group 13: 90531--91746
group 14: 94514--211559
group 15: 213921--216925
group 16: 213989--215254
group 17: 217583--224316
group 18: 222983--231134
group 19: 225008--225447
group 20: 232500--243115
group 21: 240803--241174
group 22: 243908--260361
group 23: 258539--259466
group 24: 261524--265221
group 25: 263972--270030
group 26: 272714--282265
group 27: 275802--275970
group 28: 275819--276156
group 29: 284205--288930
group 30: 284256--285758
group 31: 289656--293352
group 32: 294617--295326
group 33: 295326--295529
group 34: 298482--301317
group 35: 300895--305147
group 36: 306008--307037
group 37: 306514--306531
group 38: 312775--320352
group 39: 321060--337800
group 40: 330072--330477
group 41: 338779-

Freeing up some memory

In [ ]:
del   DEMO_FILE, pipeline, spacer,  audio, dz

# Whisper's Transcriptions

Installing Open AI whisper.

In [ ]:
!pip install git+https://github.com/openai/whisper.git

Run whisper on all audio files. Whisper generates the transcription and writes it to a file.

In [ ]:
import whisper
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = whisper.load_model('large', device = device)


In [ ]:
!pip uninstall triton
!pip install triton

Found existing installation: triton 3.0.0
Uninstalling triton-3.0.0:
  Would remove:
    /usr/local/bin/proton
    /usr/local/bin/proton-viewer
    /usr/local/lib/python3.10/dist-packages/triton-3.0.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/triton/*
Proceed (Y/n)? ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3109, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2902, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 180, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req

In [ ]:
import json
import whisper
import math

results = []

for i in range(len(groups)):
  audiof = str(i) + '.wav'

  """
   If the length of groups equals 1, this means there is only one speaker present and the result will transcribe in the form of one output.
   This can be an issue especially for larger files as even with the timestamps given, the user can't know whether the info given is
   accurate or not until the full audio is read and transcribed.

   With the section duration, the user can edit how sectioned they want the audio in the form of seconds. This can help if the desire
   is to separate the audio by minutes, hours, etc.

   Additionally, for groups of multiple speakers, the audio is also printed to show what the speakers are saying
  """
  if(len(groups) == 1):
    audio = AudioSegment.from_wav(audiof)

    # Calculate the length of the audio file and find the output of each section
    duration = len(audio)

    # Find the section duration and have it so each 10 seconds is printed
    sectionDuration = 10 * 805

    for i in range(0, duration, sectionDuration):

      # Create a new audio segment for the specific 10 second section
      part = audio[i:i+sectionDuration]

      # Export the audio segment to a usable file
      part.export('part1.wav', format='wav')

      # Transcribe the section individually to determine optimal
      tempResult = model.transcribe(audio="part1.wav", language='en', word_timestamps=True)

      # Print the results to establish the model is working as intended as well as determine
      print(tempResult["text"])
      results.append(tempResult)

  result = model.transcribe(audio=audiof, language='en', word_timestamps=True)#, initial_prompt=result.get('text', ""))
  print(result["text"])
  with open(str(i)+'.json', "w") as outfile:
    json.dump(result, outfile, indent=4)

 All right, we are recording.
 Well, good morning everybody. My name is Carvel Jones. I'll be doing the interview portion today with Mr. Ron Burt and Ms. Steph. I didn't get your last name. Excuse me, Hamblin. Hamblin. Today as we go through this E4B PDM conversation in the room, we have Mr. Chris Reese, we have Lauren here, we have Chris Scott Nicholson, and we have Captain Butler with us as well. So, let's get started.
 Hamlet.
 Go.

 I'll proceed with the questions and if anybody online has anything they want to jump in, just pop in and let us know and we'll continue the process. Okay, so let's give you a quick disclaimer. So the interview questions for this E4B program, event reviews within the PDM process, there are tasks associated with the modifications. In our discussions, we're looking to isolate the PDM process and include modifications where appropriate. So, first question we have is, we'll start with you, Steph. Can you walk me through the overarching steps you follow durin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Generating the HTML and/or txt file from the Transcriptions and the Diarization

Change or add to the speaker names and collors bellow as you wish `(speaker, textbox color, speaker color)`.

In [ ]:
speakers = {'SPEAKER_00':('SPEAKER_00', '#e1ffc7', 'darkgreen'), 'SPEAKER_01':('SPEAKER_01', 'white', 'darkorange'), 'SPEAKER_05':('Carvel Jones', 'white', 'darkorange')}
def_boxclr = 'white'
def_spkrclr = 'orange'

In the generated HTML,  the transcriptions for each diarization group are written in a box, with the speaker name on the top. By clicking a transcription, the embedded video jumps to the right time .

In [ ]:
if Source == 'Youtube':
    preS = '<!DOCTYPE html>\n<html lang="en">\n\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\n\t<title>' + \
video_title+ \
'</title>\n\t<style>\n\t\tbody {\n\t\t\tfont-family: sans-serif;\n\t\t\tfont-size: 14px;\n\t\t\tcolor: #111;\n\t\t\tpadding: 0 0 1em 0;\n\t\t\tbackground-color: #efe7dd;\n\t\t}\n\n\t\ttable {\n\t\t\tborder-spacing: 10px;\n\t\t}\n\n\t\tth {\n\t\t\ttext-align: left;\n\t\t}\n\n\t\t.lt {\n\t\t\tcolor: inherit;\n\t\t\ttext-decoration: inherit;\n\t\t}\n\n\t\t.l {\n\t\t\tcolor: #050;\n\t\t}\n\n\t\t.s {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t.c {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t.e {\n\t\t\t/*background-color: white; Changing background color */\n\t\t\tborder-radius: 10px;\n\t\t\t/* Making border radius */\n\t\t\twidth: 50%;\n\t\t\t/* Making auto-sizable width */\n\t\t\tpadding: 0 0 0 0;\n\t\t\t/* Making space around letters */\n\t\t\tfont-size: 14px;\n\t\t\t/* Changing font size */\n\t\t\tmargin-bottom: 0;\n\t\t}\n\n\t\t.t {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t#player-div {\n\t\t\tposition: sticky;\n\t\t\ttop: 20px;\n\t\t\tfloat: right;\n\t\t\twidth: 40%\n\t\t}\n\n\t\t#player {\n\t\t\taspect-ratio: 16 / 9;\n\t\t\twidth: 100%;\n\t\t\theight: auto;\n\n\t\t}\n\n\t\ta {\n\t\t\tdisplay: inline;\n\t\t}\n\t</style>\n\t<script>\n\t\tvar tag = document.createElement(\'script\');\n\t\ttag.src = "https://www.youtube.com/iframe_api";\n\t\tvar firstScriptTag = document.getElementsByTagName(\'script\')[0];\n\t\tfirstScriptTag.parentNode.insertBefore(tag, firstScriptTag);\n\t\tvar player;\n\t\tfunction onYouTubeIframeAPIReady() {\n\t\t\tplayer = new YT.Player(\'player\', {\n\t\t\t\t//height: \'210\',\n\t\t\t\t//width: \'340\',\n\t\t\t\tvideoId: \''+ \
video_id + \
'\',\n\t\t\t});\n\n\n\n\t\t\t// This is the source "window" that will emit the events.\n\t\t\tvar iframeWindow = player.getIframe().contentWindow;\n\t\t\tvar lastword = null;\n\n\t\t\t// So we can compare against new updates.\n\t\t\tvar lastTimeUpdate = "-1";\n\n\t\t\t// Listen to events triggered by postMessage,\n\t\t\t// this is how different windows in a browser\n\t\t\t// (such as a popup or iFrame) can communicate.\n\t\t\t// See: https://developer.mozilla.org/en-US/docs/Web/API/Window/postMessage\n\t\t\twindow.addEventListener("message", function (event) {\n\t\t\t\t// Check that the event was sent from the YouTube IFrame.\n\t\t\t\tif (event.source === iframeWindow) {\n\t\t\t\t\tvar data = JSON.parse(event.data);\n\n\t\t\t\t\t// The "infoDelivery" event is used by YT to transmit any\n\t\t\t\t\t// kind of information change in the player,\n\t\t\t\t\t// such as the current time or a playback quality change.\n\t\t\t\t\tif (\n\t\t\t\t\t\tdata.event === "infoDelivery" &&\n\t\t\t\t\t\tdata.info &&\n\t\t\t\t\t\tdata.info.currentTime\n\t\t\t\t\t) {\n\t\t\t\t\t\t// currentTime is emitted very frequently (milliseconds),\n\t\t\t\t\t\t// but we only care about whole second changes.\n\t\t\t\t\t\tvar ts = (data.info.currentTime).toFixed(1).toString();\n\t\t\t\t\t\tts = (Math.round((data.info.currentTime) * 5) / 5).toFixed(1);\n\t\t\t\t\t\tts = ts.toString();\n\t\t\t\t\t\tconsole.log(ts)\n\t\t\t\t\t\tif (ts !== lastTimeUpdate) {\n\t\t\t\t\t\t\tlastTimeUpdate = ts;\n\n\t\t\t\t\t\t\t// It\'s now up to you to format the time.\n\t\t\t\t\t\t\t//document.getElementById("time2").innerHTML = time;\n\t\t\t\t\t\t\tword = document.getElementById(ts)\n\t\t\t\t\t\t\tif (word) {\n\t\t\t\t\t\t\t\tif (lastword) {\n\t\t\t\t\t\t\t\t\tlastword.style.fontWeight = \'normal\';\n\t\t\t\t\t\t\t\t}\n\t\t\t\t\t\t\t\tlastword = word;\n\t\t\t\t\t\t\t\t//word.style.textDecoration = \'underline\';\n\t\t\t\t\t\t\t\tword.style.fontWeight = \'bold\';\n\n\t\t\t\t\t\t\t\tlet toggle = document.getElementById("autoscroll");\n\t\t\t\t\t\t\t\tif (toggle.checked) {\n\t\t\t\t\t\t\t\t\tlet position = word.offsetTop - 20;\n\t\t\t\t\t\t\t\t\twindow.scrollTo({\n\t\t\t\t\t\t\t\t\t\ttop: position,\n\t\t\t\t\t\t\t\t\t\tbehavior: \'smooth\'\n\t\t\t\t\t\t\t\t\t});\n\t\t\t\t\t\t\t\t}\n\n\t\t\t\t\t\t\t}\n\t\t\t\t\t\t}\n\t\t\t\t\t}\n\t\t\t\t}\n\t\t\t})\n\t\t}\n\t\tfunction jumptoTime(timepoint, id) {\n\t\t\tevent.preventDefault();\n\t\t\thistory.pushState(null, null, "#" + id);\n\t\t\tplayer.seekTo(timepoint);\n\t\t\tplayer.playVideo();\n\t\t}\n\t</script>\n</head>\n\n<body>\n\t<h2>'  + \
video_title + \
'</h2>\n\t<i>Click on a part of the transcription, to jump to its video, and get an anchor to it in the address\n\t\tbar<br><br></i>\n\t<div id="player-div">\n\t\t<div id="player"></div>\n\t\t<div><label for="autoscroll">auto-scroll: </label>\n\t\t\t<input type="checkbox" id="autoscroll" checked>\n\t\t</div>\n\t</div>\n  '
else:
    preS = '\n<!DOCTYPE html>\n<html lang="en">\n\n<head>\n\t<meta charset="UTF-8">\n\t<meta name="viewport" content="whtmlidth=device-width, initial-scale=1.0">\n\t<meta http-equiv="X-UA-Compatible" content="ie=edge">\n\t<title>' + \
    audio_title+ \
    '</title>\n\t<style>\n\t\tbody {\n\t\t\tfont-family: sans-serif;\n\t\t\tfont-size: 14px;\n\t\t\tcolor: #111;\n\t\t\tpadding: 0 0 1em 0;\n\t\t\tbackground-color: #efe7dd;\n\t\t}\n\n\t\ttable {\n\t\t\tborder-spacing: 10px;\n\t\t}\n\n\t\tth {\n\t\t\ttext-align: left;\n\t\t}\n\n\t\t.lt {\n\t\t\tcolor: inherit;\n\t\t\ttext-decoration: inherit;\n\t\t}\n\n\t\t.l {\n\t\t\tcolor: #050;\n\t\t}\n\n\t\t.s {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t.c {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t.e {\n\t\t\t/*background-color: white; Changing background color */\n\t\t\tborder-radius: 10px;\n\t\t\t/* Making border radius */\n\t\t\twidth: 50%;\n\t\t\t/* Making auto-sizable width */\n\t\t\tpadding: 0 0 0 0;\n\t\t\t/* Making space around letters */\n\t\t\tfont-size: 14px;\n\t\t\t/* Changing font size */\n\t\t\tmargin-bottom: 0;\n\t\t}\n\n\t\t.t {\n\t\t\tdisplay: inline-block;\n\t\t}\n\n\t\t#player-div {\n\t\t\tposition: sticky;\n\t\t\ttop: 20px;\n\t\t\tfloat: right;\n\t\t\twidth: 40%\n\t\t}\n\n\t\t#player {\n\t\t\taspect-ratio: 16 / 9;\n\t\t\twidth: 100%;\n\t\t\theight: auto;\n\t\t}\n\n\t\ta {\n\t\t\tdisplay: inline;\n\t\t}\n\t</style>';
    preS += '\n\t<script>\n\twindow.onload = function () {\n\t\t\tvar player = document.getElementById("audio_player");\n\t\t\tvar player;\n\t\t\tvar lastword = null;\n\n\t\t\t// So we can compare against new updates.\n\t\t\tvar lastTimeUpdate = "-1";\n\n\t\t\tsetInterval(function () {\n\t\t\t\t// currentTime is checked very frequently (1 millisecond),\n\t\t\t\t// but we only care about whole second changes.\n\t\t\t\tvar ts = (player.currentTime).toFixed(1).toString();\n\t\t\t\tts = (Math.round((player.currentTime) * 5) / 5).toFixed(1);\n\t\t\t\tts = ts.toString();\n\t\t\t\tconsole.log(ts);\n\t\t\t\tif (ts !== lastTimeUpdate) {\n\t\t\t\t\tlastTimeUpdate = ts;\n\n\t\t\t\t\t// Its now up to you to format the time.\n\t\t\t\t\tword = document.getElementById(ts)\n\t\t\t\t\tif (word) {\n\t\t\t\t\t\tif (lastword) {\n\t\t\t\t\t\t\tlastword.style.fontWeight = "normal";\n\t\t\t\t\t\t}\n\t\t\t\t\t\tlastword = word;\n\t\t\t\t\t\t//word.style.textDecoration = "underline";\n\t\t\t\t\t\tword.style.fontWeight = "bold";\n\n\t\t\t\t\t\tlet toggle = document.getElementById("autoscroll");\n\t\t\t\t\t\tif (toggle.checked) {\n\t\t\t\t\t\t\tlet position = word.offsetTop - 20;\n\t\t\t\t\t\t\twindow.scrollTo({\n\t\t\t\t\t\t\t\ttop: position,\n\t\t\t\t\t\t\t\tbehavior: "smooth"\n\t\t\t\t\t\t\t});\n\t\t\t\t\t\t}\n\t\t\t\t\t}\n\t\t\t\t}\n\t\t\t}, 0.1);\n\t\t}\n\n\t\tfunction jumptoTime(timepoint, id) {\n\t\t\tvar player = document.getElementById("audio_player");\n\t\t\thistory.pushState(null, null, "#" + id);\n\t\t\tplayer.pause();\n\t\t\tplayer.currentTime = timepoint;\n\t\t\tplayer.play();\n\t\t}\n\t\t</script>\n\t</head>';
    preS += '\n\n<body>\n\t<h2>' + audio_title + '</h2>\n\t<i>Click on a part of the transcription, to jump to its portion of audio, and get an anchor to it in the address\n\t\tbar<br><br></i>\n\t<div id="player-div">\n\t\t<div id="player">\n\t\t\t<audio controls="controls" id="audio_player">\n\t\t\t\t<source src="input.wav" />\n\t\t\t</audio>\n\t\t</div>\n\t\t<div><label for="autoscroll">auto-scroll: </label>\n\t\t\t<input type="checkbox" id="autoscroll" checked>\n\t\t</div>\n\t</div>\n';

postS = '\t</body>\n</html>'

In [ ]:
#import webvtt
import json
from datetime import timedelta

def timeStr(t):
  return '{0:02d}:{1:02d}:{2:06.2f}'.format(round(t // 3600),
                                                round(t % 3600 // 60),
                                                t % 60)

html = list(preS)
txt = list("")
gidx = -1
for g in groups:
  shift = re.findall('[0-9]+:[0-9]+:[0-9]+\.[0-9]+', string=g[0])[0]
  shift = millisec(shift) - spacermilli #the start time in the original video
  shift=max(shift, 0)

  gidx += 1

  captions = json.load(open(str(gidx) + '.json'))['segments']

  if captions:
    speaker = g[0].split()[-1]
    boxclr = def_boxclr
    spkrclr = def_spkrclr
    if speaker in speakers:
      speaker, boxclr, spkrclr = speakers[speaker]

    html.append(f'<div class="e" style="background-color: {boxclr}">\n');
    html.append('<p  style="margin:0;padding: 5px 10px 10px 10px;word-wrap:normal;white-space:normal;">\n')
    html.append(f'<span style="color:{spkrclr};font-weight: bold;">{speaker}</span><br>\n\t\t\t\t')

    for c in captions:
      start = shift + c['start'] * 1000.0
      start = start / 1000.0   #time resolution ot youtube is Second.
      end = (shift + c['end'] * 1000.0) / 1000.0
      txt.append(f'[{timeStr(start)} --> {timeStr(end)}] [{speaker}] {c["text"]}\n')

      for i, w in enumerate(c['words']):
        if w == "":
           continue
        start = (shift + w['start']*1000.0) / 1000.0
        #end = (shift + w['end']) / 1000.0   #time resolution ot youtube is Second.
        html.append(f'<a href="#{timeStr(start)}" id="{"{:.1f}".format(round(start*5)/5)}" class="lt" onclick="jumptoTime({int(start)}, this.id)">{w["word"]}</a><!--\n\t\t\t\t-->')
    #html.append('\n')
    html.append('</p>\n')
    html.append(f'</div>\n')

html.append(postS)


with open(f"capspeaker.txt", "w", encoding='utf-8') as file:
  s = "".join(txt)
  file.write(s)
  print('captions saved to capspeaker.txt:')
  print(s+'\n')

with open(f"capspeaker.html", "w", encoding='utf-8') as file:    #TODO: proper html embed tag when video/audio from file
  s = "".join(html)
  file.write(s)
  print('captions saved to capspeaker.html:')
  print(s+'\n')

captions saved to capspeaker.txt:
[00:00:001.79 --> 00:00:002.81] [SPEAKER_04]  All right, we are recording.
[00:00:004.08 --> 00:00:007.74] [Carvel Jones]  Well, good morning everybody. My name is Carvel Jones. I'll be doing the interview portion
[00:00:007.74 --> 00:00:011.60] [Carvel Jones]  today with Mr. Ron Burt and Ms. Steph. I didn't get your last name.
[00:00:012.02 --> 00:00:012.92] [Carvel Jones]  Excuse me, Hamblin.
[00:00:013.04 --> 00:00:021.12] [Carvel Jones]  Hamblin. Today as we go through this E4B PDM conversation in the room, we have Mr. Chris
[00:00:021.12 --> 00:00:027.36] [Carvel Jones]  Reese, we have Lauren here, we have Chris Scott Nicholson, and we have Captain Butler
[00:00:027.36 --> 00:00:028.14] [Carvel Jones]  with us as well.
[00:00:028.14 --> 00:00:028.72] [Carvel Jones]  So, let's get started.
[00:00:012.70 --> 00:00:012.84] [SPEAKER_04]  Hamlet.
[00:00:024.52 --> 00:00:024.56] [SPEAKER_07]  Go.
[00:00:024.57 --> 00:00:024.57] [SPEAKER_06] 
[00:00:029.